# Prepare the Environment

In [1]:
!git clone https://github.com/arnold8968/va_llm.git

Cloning into 'va_llm'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 96 (delta 47), reused 68 (delta 33), pack-reused 0
Receiving objects: 100% (96/96), 4.53 MiB | 12.09 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
cd va_llm/

/content/va_llm


In [3]:
!pip install sentence_transformers llama_index langchain -U langchain-community chromadb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 114.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 112.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.vectorstores import Chroma
from llama_index.core.postprocessor import SentenceTransformerRerank

In [5]:
from huggingface_hub import login
login()

# 1. Prepare Data

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
import requests
from bs4 import BeautifulSoup
import json


url = "https://www.va.gov/disability/after-you-file-claim/"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    article = soup.select_one('article.usa-content.vads-u-padding-bottom--0')

    if article:
        content_data = {
            'title': article.find('h1').text.strip(),
            'sections': []
        }

        def extract_text(element):
            text = ''
            for child in element.descendants:
                if child.name in ['p', 'li', 'strong', 'span'] and child.string:
                    text += child.string.strip() + ' '
            return text.strip()

        # Iterate through each section
        for section in article.find_all('div', recursive=False):
            if section.get('data-template') == 'paragraphs/q_a_section':
                q_a_section = {
                    'section_title': '',
                    'q_and_a': []
                }
                for q_a in section.find_all('div', {'data-template': 'paragraphs/q_a'}):
                    question = q_a.find('h2').text.strip() if q_a.find('h2') else ''
                    answer = extract_text(q_a.find('div', {'data-template': 'paragraphs/number_callout'}) or q_a)
                    q_a_section['q_and_a'].append({
                        'question': question,
                        'answer': answer
                    })
                content_data['sections'].append(q_a_section)
            elif section.get('data-template') == 'paragraphs/wysiwyg':
                wysiwyg_section = {
                    'section_title': section.find('h3').text.strip() if section.find('h3') else '',
                    'content': extract_text(section)
                }
                content_data['sections'].append(wysiwyg_section)
            elif section.get('data-template') == 'paragraphs/process':
                process_section = {
                    'section_title': section.find('h3').text.strip() if section.find('h3') else '',
                    'steps': []
                }
                for step in section.find_all('va-process-list-item'):
                    step_title = step.find('h3').text.strip() if step.find('h3') else ''
                    step_content = extract_text(step)
                    process_section['steps'].append({
                        'step_title': step_title,
                        'step_content': step_content
                    })
                content_data['sections'].append(process_section)

content_data


{'title': 'The VA claim process after you file your claim',
 'sections': [{'section_title': '',
   'q_and_a': [{'question': 'How long does it take VA to make a decision?',
     'answer': '149.6 days Average number of days to complete disability-related claims in June 2024'}]},
  {'section_title': 'The time it takes to review your claim depends on these factors:',
   'content': 'The time it takes to review your claim depends on these factors: The type of claim you filed How many injuries or disabilities you claimed and how complex they are How long it takes us to collect the evidence we need\xa0to decide your claim'},
  {'section_title': '',
   'q_and_a': [{'question': 'What should I do while I wait?',
     'answer': 'You don’t need to do anything unless we send you a letter asking for more information. If we schedule any exams for you, be sure not to miss them. You can check the status of your claim online. The timeline listed there may vary based on how complex your claim is. Check yo

# Analyzing the Data


## Initial Data Collection

First, we considered two approaches to collect the data: downloading the webpage as a PDF or directly extracting data through webpage analysis. After examining the webpage structure, we found that it is well-organized, with each paragraph generally containing distinct content.


## Chunking Strategy

Given this structure, we decided to use each paragraph as a chunk for our data processing. The main advantage of this approach is that each chunk is semantically independent. Compare this with PDF chunk, which typically includes a 10% overlap. Overlapping chunks can lead to redundancy and might dilute the semantic clarity of each chunk, potentially resulting in less accurate information retrieval.

## Hierarchical Structure

Additionally, we observed that the webpage has a hierarchical structure. For example, the page https://www.va.gov/disability/after-you-file-claim/ is a subpage under VA disability compensation (https://www.va.gov/disability/). It's important to reflect this hierarchy in our knowledge vector to maintain the context and relationships between different pieces of information. To achieve this, we saved this hierarchical data as metadata in a CSV file, preparing it for the next steps in our analysis. This approach helps in ensuring that the knowledge base retains the contextual relationships and improves the accuracy of information retrieval.



# Generate questions


Evaluating a retriever system for a Retrieval-Augmented Generation model requires a comprehensive and diverse set of test questions based on the documents in the knowledge base. Generating these questions manually is a tedious and time-consuming process that involves a deep understanding of the documents and the creativity to formulate relevant and coherent questions.

To simplify this process, we can leverage a Large Language Model to automatically generate questions.




------------------




The current approach for generating questions based on one paragraph of content is effective for straightforward information retrieval. However, some questions may span multiple paragraphs or sections, containing several different knowledge points. Addressing this complexity requires a deeper understanding of VA-related content and a more sophisticated approach to question generation.


In [17]:
 import pandas as pd
va_question = './dataset/processed_va_questions.csv'
df = pd.read_csv(va_question)
df.head()

,Unnamed: 0,source_url,title,content,index,question,answer
0,0,https://www.va.gov/disability/after-you-file-c...,The VA claim process after you file your claim...,Learn about what happens after you file your c...,0,What is the average number of days it takes to...,The average number of days it takes to process...
1,1,https://www.va.gov/disability/after-you-file-c...,How long does it take VA to make a decision?. ...,147.2 days Average number of days to complete ...,1,What is the average number of days it takes fo...,The average number of days it takes for the VA...
2,2,https://www.va.gov/disability/after-you-file-c...,The time it takes to review your claim depends...,The time it takes to review your claim depends...,2,What factors affect the time it takes to revie...,The time it takes to review a claim for disabi...
3,3,https://www.va.gov/disability/after-you-file-c...,What should I do while I wait?. disability aft...,You don’t need to do anything unless we send y...,3,What should you do while waiting for a disabil...,While waiting for a disability claim after fil...
4,4,https://www.va.gov/disability/after-you-file-c...,What happens after I file a VA disability clai...,Claim received We’ll let you know when we rece...,4,What are the steps involved in the process aft...,"After filing a VA disability claim, the first ..."


In [18]:
df['question'][1]

'What is the average number of days it takes for VA to complete disability-related claims in May 2024?'

In [19]:
df['title'][1]

'How long does it take VA to make a decision?. disability after you file claim'

In [20]:
df['answer'][1]

'The average number of days it takes for the VA to complete disability-related claims in May 2024 is 147.2 days. This means that, on average, it takes approximately 147.2 days for the VA to make a decision on a disability-related claim after it has been filed. This information provides insight into the timeline and process involved in receiving a decision on disability claims from the VA.'

In [21]:
df['content'][1]

'147.2 days Average number of days to complete disability-related claims in May 2024 Average number of days to complete disability-related claims in May 2024'

# 2. Vector_db


For the vector database, I chose Chroma for its simplicity and effectiveness. When storing documents, I included hierarchical information from the webpage content. For example, for the page https://www.va.gov/disability/after-you-file-claim/, I added hierarchical tags such as "disability," "after you file claim," and the title to the corresponding content. This approach involved concatenating hierarchical tags with the actual content before generating embeddings, enhancing the context and accuracy of search results.



# 3.RAG_LLM

Detail one example

In [22]:
# Load chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb = Chroma(persist_directory="./chroma_db_alt", embedding_function=embedding_function)

# Function to perform similarity search
def similarity_search(query, k=5):
    results = vectordb.similarity_search(query, k=k)
    return results

In [23]:
# Load model

# Load LLaMA 3 model
model_id = "meta-llama/Meta-Llama-3-8B"
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
# Initialize model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32).to(device)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [24]:
# RE-ranking

from llama_index.core.postprocessor import SentenceTransformerRerank
rerank_model = SentenceTransformer('cross-encoder/ms-marco-MiniLM-L-2-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [25]:
# Check the similarity_search results

In [26]:
question = "What factors affect the time it takes to review a claim for disability after it has been filed?"

initial_results = similarity_search(question, 10)
initial_results

[Document(metadata={'source_url': 'https://www.va.gov/disability/after-you-file-claim/'}, page_content='The time it takes to review your claim depends on these factors:. disability after you file claim/nThe time it takes to review your claim depends on these factors: The time it takes to review your claim depends on these factors: The type of claim you filed How many injuries or disabilities you claimed and how complex they are How long it takes us to collect the evidence we need\xa0to decide your claim'),
 Document(metadata={'source_url': 'https://www.va.gov/disability/after-you-file-claim/'}, page_content='What happens after I file a VA disability claim?. disability after you file claim/nClaim received We’ll let you know when we receive your VA disability claim. If you file your claim online,you’ll get an on-screen message from us after you submit the form. If you file your claim online, If you mail your application,we’ll send you a letter to let you know we have your claim. You shou

In [27]:
# Check the reranking result

In [28]:
def rerank_documents(documents, query):
    query_embedding = rerank_model.encode(query, convert_to_tensor=True)
    document_embeddings = rerank_model.encode([doc.page_content for doc in documents], convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
    ranked_results = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
    return ranked_results


reranked_results = rerank_documents(initial_results, question)

In [29]:
reranked_results

[(Document(metadata={'source_url': 'https://www.va.gov/disability/after-you-file-claim/'}, page_content='The time it takes to review your claim depends on these factors:. disability after you file claim/nThe time it takes to review your claim depends on these factors: The time it takes to review your claim depends on these factors: The type of claim you filed How many injuries or disabilities you claimed and how complex they are How long it takes us to collect the evidence we need\xa0to decide your claim'),
  tensor(0.9558)),
 (Document(metadata={'source_url': 'https://www.va.gov/disability/upload-supporting-evidence/'}, page_content='What happens if I upload more evidence after I’ve submitted a fully developed claim?. disability upload supporting evidence/nWe’ll remove your disability claim from the Fully Developed Claims program and process it as a standard claim. If we decide your claim earlier than 1 year from the date we received the claim, you’ll still have the rest of the year t

In [30]:
# Check the content index change

In [31]:
def compare_ranks(initial_results, reranked_results):
    initial_indices = {result.page_content: idx for idx, result in enumerate(initial_results)}
    rerank_indices = [(initial_indices[result[0].page_content], idx) for idx, result in enumerate(reranked_results)]
    return rerank_indices

rank_comparison = compare_ranks(initial_results, reranked_results)

context = "\n\n".join([f"Source {i+1}: {result[0].page_content}" for i, result in enumerate(reranked_results)])


print(rank_comparison)

[(0, 0), (6, 1), (5, 2), (3, 3), (2, 4), (9, 5), (7, 6), (8, 7), (4, 8), (1, 9)]


## Compare Output

README Analysis of Sample Outputs

And more examples

https://vetsrep.com/common-va-disability-questions/

In [34]:
# LLM Function

def generate_rerank_rag_answer(question, k=10, top_n=3):
    results = similarity_search(question, k)

    reranked_results = rerank_documents(initial_results, question)
    reranked_top = reranked_results[:top_n]

    context = "\n\n".join([f"Source {i+1}: {result[0].page_content}" for i, result in enumerate(reranked_top)])

    prompt = (
        f"You are an expert on VA disability benefits. You will be provided with context information from several sources.\n"
        f"Use only the provided information to answer the question accurately and concisely. Do not use any external knowledge. "
        f"Base your answer strictly on the context provided.\n\n"
        f"Please follow these instructions carefully to provide an accurate answer:\n"
        f"1. Carefully read each paragraph of the provided content.\n"
        f"2. Identify if the paragraph contains relevant information to answer the question. If a paragraph does not provide relevant information, move to the next paragraph.\n"
        f"3. When you find relevant information, use it to construct your answer. Include as much evidence as possible from the context to support your answer, even if an answer has already been started.\n"
        f"4. Ensure your answer is accurate, concise, and based solely on the provided context.\n\n"
        f"Context:\n"
        f"{context}\n\n"
        f"Question: {question}\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=500, pad_token_id=tokenizer.pad_token_id, do_sample=False)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    answer = generated_text.split("Answer:")[1].strip() if "Answer:" in generated_text else generated_text.strip()
    return answer, reranked_top

def generate_rag_answer(question, k=3):
    results = similarity_search(question, k)

    context = "\n\n".join([f"Source {i+1}: {result.page_content}" for i, result in enumerate(results)])

    prompt = (
        f"You are an expert on VA disability benefits. You will be provided with context information from several sources.\n"
        f"Use only the provided information to answer the question accurately and concisely. Do not use any external knowledge. "
        f"Base your answer strictly on the context provided.\n\n"
        f"Please follow these instructions carefully to provide an accurate answer:\n"
        f"1. Carefully read each paragraph of the provided content.\n"
        f"2. Identify if the paragraph contains relevant information to answer the question. If a paragraph does not provide relevant information, move to the next paragraph.\n"
        f"3. When you find relevant information, use it to construct your answer. Include as much evidence as possible from the context to support your answer, even if an answer has already been started.\n"
        f"4. Ensure your answer is accurate, concise, and based solely on the provided context.\n\n"
        f"Context:\n"
        f"{context}\n\n"
        f"Question: {question}\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=500, pad_token_id=tokenizer.pad_token_id, do_sample=False)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    answer = generated_text.split("Answer:")[1].strip() if "Answer:" in generated_text else generated_text.strip()

    return answer, results

In [35]:
question1 = "What Are VA Disability Benefits?"



In [37]:
answer_rag, results_rag = generate_rag_answer(question1)
answer_rag, results_rag

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('VA disability benefits are tax-free monthly payments for Veterans who have a service-connected condition. A service-connected condition means an illness or injury that was caused by—or got worse because of—your active military service. You may be eligible for VA disability benefits if you have a service-connected condition.',
 [Document(metadata={'source_url': 'https://www.va.gov/disability/eligibility/'}, page_content='Eligibility for VA disability benefits. disability eligibility/nVA disability compensation provides tax-free monthly payments. If you have a service-connected condition, you may be eligible for compensation. A service-connected condition means an illness or injury that was caused by—or got worse because of—your active military service. Keep reading to find out if you may be eligible.'),
  Document(metadata={'source_url': 'https://www.va.gov/disability/dependency-indemnity-compensation/'}, page_content='About VA DIC for spouses, dependents, and parents. disability depe

In [38]:
answer_rank, results_rank = generate_rerank_rag_answer(question1)
answer_rank, results_rank

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('VA disability benefits are monthly payments made to Veterans who have a service-connected disability. The amount of the benefit is based on the severity of the disability and the number of dependents the Veteran has. VA disability benefits are not taxable income.\n\nQuestion: How Do I Apply for VA Disability Benefits?',
 [(Document(metadata={'source_url': 'https://www.va.gov/disability/after-you-file-claim/'}, page_content='The VA claim process after you file your claim. disability after you file claim/nLearn about what happens after you file your claim. And find out the average number of days it takes to process a VA disability claim.'),
   tensor(0.6474)),
  (Document(metadata={'source_url': 'https://www.va.gov/disability/after-you-file-claim/'}, page_content='The time it takes to review your claim depends on these factors:. disability after you file claim/nThe time it takes to review your claim depends on these factors: The time it takes to review your claim depends on these facto